## Instalação de dependências

Esta célula instala as bibliotecas Python necessárias para executar o restante do notebook, incluindo:

*   `gensim`: Para trabalhar com modelos de word embeddings, como Word2Vec.
*   `ipykernel`: Essencial para o funcionamento do ambiente de notebook.
*   `requests`: Para fazer download de dados da internet.
*   `pandas`: Para manipulação e análise de dados em formato de tabela.
*   `matplotlib` e `seaborn`: Para visualização de dados.

Se você estiver executando este notebook no Google Colab e for solicitado a reiniciar o ambiente de execução após a instalação, por favor, faça-o para garantir que todas as bibliotecas estejam carregadas corretamente antes de prosseguir.

In [ ]:
%pip install gensim
%pip install ipykernel
%pip install requests
%pip install pandas
%pip install matplotlib
%pip install seaborn

In [ ]:
# Importação das bibliotecas necessárias
import gensim

print(f"Gensim instalado na versão: {gensim.__version__}")

## Download e Preparação dos Dados

Nesta etapa, é realizado o download e a extração dos arquivos que servirão de base para o nosso estudo. O corpus de texto `text8.zip` é um conjunto de dados utilizado para o treinamento de modelos de *word embeddings*, contendo uma grande coleção de textos.

Além do corpus, também baixamos o arquivo `questions-words.txt`, que é um recurso valioso para a avaliação da qualidade do nosso modelo Word2Vec. Este arquivo contém uma lista de analogias semânticas e sintáticas, que nos permitirá testar se o modelo aprendeu relações significativas entre as palavras.

Todos os arquivos baixados serão armazenados em uma pasta local chamada `data`, mantendo nosso projeto organizado e os dados facilmente acessíveis.

In [ ]:
import os
import requests
import zipfile

def download_and_extract(url, dest_path):
    filename = url.split('/')[-1]
    filepath = os.path.join(dest_path, filename)

    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    # Verifica se o arquivo já existe (descompactado, se for zip)
    if not os.path.exists(filepath.replace('.zip', '')):
        print(f"Baixando {filename}...")
        r = requests.get(url, stream=True)

        # Salva o arquivo em chunks
        with open(filepath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"{filename} baixado.")

        if filename.endswith('.zip'):
            print(f"Extraindo {filename}...")
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall(dest_path)
            os.remove(filepath)
            print("Extração completa.")
    else:
        print(f"{filename.replace('.zip', '')} já existe. Download pulado.")

# URLs dos arquivos que serão baixados
corpus_url = 'http://mattmahoney.net/dc/text8.zip'
analogy_url = 'https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt'
dest_folder = '../data'

# Baixar e extrair o corpus de treinamento (text8.zip)
download_and_extract(corpus_url, dest_folder)
corpus_path = os.path.join(dest_folder, 'text8')

# Baixar o arquivo de analogias (questions-words.txt)
download_and_extract(analogy_url, dest_folder)
analogy_path = os.path.join(dest_folder, 'questions-words.txt')

## Pré-processamento do Texto

Nesta seção, o texto do corpus `text8` passa por um processo de **pré-processamento** para prepará-lo para o treinamento do modelo Word2Vec. As etapas incluem:

1.  **Remoção de Stopwords:** Palavras comuns que geralmente não carregam significado semântico importante (stopwords) são removidas do texto. Isso ajuda a reduzir o ruído e focar nas palavras mais relevantes.
2.  **Detecção de Bigramas:** O modelo `gensim.models.Phrases` é utilizado para identificar e agrupar palavras que frequentemente aparecem juntas, formando "frases" ou bigramas (por exemplo, "new_york"). Isso permite que o modelo trate essas combinações como unidades únicas, capturando melhor o seu significado contextual.

O resultado dessas etapas é um corpus processado, pronto para ser usado no treinamento do modelo Word2Vec.

In [ ]:
import os
from gensim.models import Phrases
from gensim.parsing.preprocessing import remove_stopwords

corpus_path = os.path.join(dest_folder, 'text8')

# Abre o arquivo do corpus e lê todo o conteúdo, dividindo em palavras
with open(corpus_path, 'r', encoding='utf-8') as f:
    words = f.read().split()

CHUNK_SIZE = 10000

# Divide a lista de palavras em chunks e remove stopwords de cada chunk
chunked_sentences = [remove_stopwords(" ".join(words[i:i + CHUNK_SIZE])) for i in range(0, len(words), CHUNK_SIZE)]

# O modelo Phrases espera uma lista de listas de strings (sentenças divididas em palavras)
processed_chunks = [sentence.split() for sentence in chunked_sentences]

# Cria um modelo de bigramas para identificar frases comuns (ex: "new_york")
bigram_model = Phrases(processed_chunks, min_count=5, threshold=10, progress_per=100)
final_corpus = bigram_model[processed_chunks]

sentences = list(final_corpus)

print("\nPreparação de dados concluída com sucesso!")
size = sum(len(sentence) for sentence in sentences)
print(f"Corpus final pronto com {size} palavras para o treinamento.")

## Pré-processamento dos Dados de Analogia

Responsável por realiza um pré-processamento simples para preparar os dados para a avaliação `questions-words.txt` :

1.  **Leitura do Arquivo:** O arquivo é lido linha por linha.
2.  **Filtragem:** Linhas que começam com o caractere `:` são ignoradas, pois elas indicam categorias e não analogias.
3.  **Tokenização e Normalização:** Para cada linha válida, o texto é dividido em palavras (tokenização), e todas as palavras são convertidas para minúsculas. Espaços em branco extras no início ou fim da linha também são removidos.

Este pré-processamento garante que os dados de analogia estejam em um formato consistente e limpo, pronto para ser usado pela função de avaliação do modelo Word2Vec.

In [ ]:
analogy_lines = []
with open(analogy_path) as file:
    for line in file:
        # Ignora linhas que começam com ':'
        if line.startswith(':'):
            continue

        # Divide a linha em palavras, converte para minúsculas e remove espaços em branco
        line = [word.lower() for word in line.strip().split()]
        analogy_lines.append(line)

print(analogy_lines[:3])

In [ ]:
import numpy as np

def vectorized_cosine_distance(A, B):
    dot_product = np.einsum('ij,ij->i', A, B)
    norm_A = np.linalg.norm(A, axis=1)
    norm_B = np.linalg.norm(B, axis=1)
    denominator = (norm_A * norm_B) + 1e-8
    similarity = dot_product / denominator
    similarity = np.clip(similarity, -1.0, 1.0)
    return 1 - similarity

def evaluate_word_analogies(model, analogies):
    vocab = model.wv.key_to_index

    valid_analogies = [
        line for line in analogies
        if all(word in vocab for word in line)
    ]

    if not valid_analogies:
        return 0.0

    valid_analogies_np = np.array(valid_analogies)

    vecs1 = model.wv[valid_analogies_np[:, 0]] # Vetor A
    vecs2 = model.wv[valid_analogies_np[:, 1]] # Vetor B
    vecs3 = model.wv[valid_analogies_np[:, 2]] # Vetor C

    predicted_vecs = vecs2 - vecs1 + vecs3

    expected_vecs = model.wv[valid_analogies_np[:, 3]] # Vetor D (esperado)
    distances = vectorized_cosine_distance(predicted_vecs, expected_vecs)

    return np.mean(distances)

In [ ]:
import itertools
import pandas as pd

# Definindo os hiperparâmetros para o experimento
param_grid = {
    'sg': [0, 1],  # 0 para CBOW, 1 para Skip-gram
    'vector_size': [250, 300, 350], # Tamanho do embedding
    'window': [5, 7, 10], # Janela de contexto
    'epochs': [5, 10, 20, 30], # Iterações de treinamento
    'alpha': [0.05, 0.025, 0.01, 0.0075] # Taxa de aprendizado inicial
}

# Gerar todas as combinações de parâmetros
param_combinations = list(itertools.product(
    param_grid['sg'],
    param_grid['vector_size'],
    param_grid['window'],
    param_grid['epochs'],
    param_grid['alpha']
))

print(f"Total de combinações de parâmetros: {len(param_combinations)}")

In [ ]:
from gensim.models import Word2Vec
import time
import os

print("Iniciando o treinamento e avaliação dos modelos...")
print("="*50)

# Semente utilizada para ter resultado comparáveis
seed = 137
cores = os.cpu_count() or 1
model_count = 0
total_models = len(param_combinations)

# Define a base path for models
model_base_path = os.path.join(dest_folder, 'word2vec_models')

# Create the models directory if it doesn't exist
if not os.path.exists(model_base_path):
    os.makedirs(model_base_path)

results = []

for sg, size, window, epochs, alpha in param_combinations:
    model_count += 1
    start_time = time.time()

    model_name = f"sg={sg}_size={size}_window={window}_epochs={epochs}_alpha={alpha}"
    model_path = os.path.join(model_base_path, model_name)
    architecture_name = 'Skip-gram' if sg == 1 else 'CBOW'

    print(f"({model_count}/{total_models}) Processando modelo: {architecture_name}, Vec Size={size}, Window={window}, Epochs={epochs}, Alpha={alpha}")

    # Check if the model already exists
    if os.path.exists(model_path):
        print(f"Carregando modelo existente de: {model_path}")
        model = Word2Vec.load(model_path)
    else:
        print(f"Treinando novo modelo: {architecture_name}, Vec Size={size}, Window={window}, Epochs={epochs}, Alpha={alpha}")
        model = Word2Vec(
            sentences=sentences,
            sg=sg,
            vector_size=size,
            window=window,
            epochs=epochs,
            min_count=5, # Ignora palavras com frequência menor que 5
            workers=cores,
            alpha=alpha,
            seed=seed
        )
        print("Treinamento concluído. Salvando modelo...")
        model.save(model_path)

    print("Iniciando avaliação de analogias...")
    mean_distance = evaluate_word_analogies(model, analogy_lines)
    print("Distância média:", mean_distance)

    end_time = time.time()
    duration = end_time - start_time
    print(f"Duração do processo: {duration:.2f} segundos")

    print("-"*50)

    results.append({
        'Arquitetura': architecture_name,
        'Tamanho Embedding': size,
        'Janela Contexto': window,
        'Épocas': epochs,
        'Taxa Aprendizado': alpha,
        'Distância média': mean_distance,
        'Duração (s)': duration
    })

print("Todos os modelos foram processados e avaliados com sucesso!")

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
best_models = results_df.sort_values(by='Distância média')

display(best_models)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configura o estilo dos gráficos
sns.set_theme(style="whitegrid")

# Gráfico 1: Comparação de Arquitetura (CBOW vs. Skip-gram)
plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x='Arquitetura', y='Distância média')
plt.title('Distância Média por Arquitetura do Modelo')
plt.ylabel('Distância Média de Analogia')
plt.xlabel('Arquitetura')
plt.show()

# Gráfico 2: Impacto do Tamanho do Embedding
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df, x='Tamanho Embedding', y='Distância média', hue='Arquitetura', style='Arquitetura', markers=True, dashes=False)
plt.title('Impacto do Tamanho do Embedding na Distância Média')
plt.ylabel('Distância Média')
plt.xlabel('Dimensões do Vetor (Embedding Size)')
plt.legend(title='Arquitetura')
plt.show()

# Gráfico 3: Impacto do Tamanho da Janela de Contexto
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df, x='Janela Contexto', y='Distância média', hue='Arquitetura', style='Arquitetura', markers=True, dashes=False)
plt.title('Impacto da Janela de Contexto na Distância Média')
plt.ylabel('Distância Média')
plt.xlabel('Tamanho da Janela')
plt.legend(title='Arquitetura')
plt.show()

# Gráfico 4: Impacto do Número de Épocas
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df, x='Épocas', y='Distância média', hue='Arquitetura', style='Arquitetura', markers=True, dashes=False)
plt.title('Impacto do Número de Épocas na Distância Média')
plt.ylabel('Distância Média')
plt.xlabel('Iterações de Treinamento')
plt.legend(title='Arquitetura')
plt.show()

# Gráfico 5: Impacto da Taxa de Aprendizado
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df, x='Taxa Aprendizado', y='Distância média', hue='Arquitetura', style='Arquitetura', markers=True, dashes=False)
plt.title('Impacto da Taxa de Aprendizado na Distância Média')
plt.ylabel('Distância Média')
plt.xlabel('Taxa de Aprendizado Inicial (Alpha)')
plt.legend(title='Arquitetura')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráfico 6: Interação entre Taxa de Aprendizado (Alpha) e Épocas
g = sns.catplot(
    data=results_df,
    x='Épocas',
    y='Distância média',
    hue='Taxa Aprendizado',
    col='Arquitetura',
    kind='point',
    palette='flare',
    height=6,
    aspect=1.2
)
g.fig.suptitle('Impacto Combinado de Épocas e Taxa de Aprendizado', y=1.03)
g.set_axis_labels('Número de Épocas', 'Distância Média')
g.legend.set_title('Taxa de Aprendizado')
plt.show()

# Gráfico 7: Interação entre Tamanho do Embedding e Janela de Contexto
g = sns.catplot(
    data=results_df,
    x='Tamanho Embedding',
    y='Distância média',
    hue='Janela Contexto',
    col='Arquitetura',
    kind='point',
    palette='flare',
    height=6,
    aspect=1.2
)
g.fig.suptitle('Impacto Combinado do Tamanho do Embedding e Janela de Contexto', y=1.03)
g.set_axis_labels('Tamanho do Embedding', 'Distância Média')
g.legend.set_title('Janela de Contexto')
plt.show()

# Gráfico 8: Interação entre Épocas e Janela de Contexto
g = sns.catplot(
    data=results_df,
    x='Épocas',
    y='Distância média',
    hue='Janela Contexto',
    col='Arquitetura',
    kind='point',
    palette='flare',
    height=6,
    aspect=1.2
)
g.fig.suptitle('Impacto Combinado de Épocas e Janela de Contexto', y=1.03)
g.set_axis_labels('Número de Épocas', 'Distância Média')
g.legend.set_title('Janela de Contexto')
plt.show()

# Gráfico 9: Interação entre Tamanho do Embedding e Taxa de Aprendizado
g = sns.catplot(
    data=results_df,
    x='Tamanho Embedding',
    y='Distância média',
    hue='Taxa Aprendizado',
    col='Arquitetura',
    kind='point',
    palette='flare',
    height=6,
    aspect=1.2
)
g.fig.suptitle('Impacto Combinado do Embedding e Taxa de Aprendizado', y=1.03)
g.set_axis_labels('Tamanho do Embedding', 'Distância Média')
g.legend.set_title('Taxa de Aprendizado')
plt.show()

# Gráfico 10: Interação entre Janela de Contexto e Taxa de Aprendizado
g = sns.catplot(
    data=results_df,
    x='Janela Contexto',
    y='Distância média',
    hue='Taxa Aprendizado',
    col='Arquitetura',
    kind='point',
    palette='flare',
    height=6,
    aspect=1.2
)
g.fig.suptitle('Impacto Combinado da Janela de Contexto e Taxa de Aprendizado', y=1.03)
g.set_axis_labels('Janela de Contexto', 'Distância Média')
g.legend.set_title('Taxa de Aprendizado')
plt.show()

# Gráfico 11: Interação entre Épocas e Tamanho do Embedding
g = sns.catplot(
    data=results_df,
    x='Épocas',
    y='Distância média',
    hue='Tamanho Embedding',
    col='Arquitetura',
    kind='point',
    palette='flare',
    height=6,
    aspect=1.2
)
g.fig.suptitle('Impacto Combinado de Épocas e Tamanho do Embedding', y=1.03)
g.set_axis_labels('Número de Épocas', 'Distância Média')
g.legend.set_title('Tamanho Embedding')
plt.show()